In [1]:
import os
import yaml
import random

import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image

import matplotlib.pyplot as plt

LABELS_PATH="labels.yaml"

2023-05-29 20:58:17.316614: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-29 20:58:17.993278: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/cuda-11.3/targets/x86_64-linux/lib
2023-05-29 20:58:17.993362: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidi

In [2]:
model = tf.keras.models.load_model('output')
model.summary()

2023-05-29 20:58:18.980891: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-29 20:58:18.985738: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-29 20:58:18.985917: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-29 20:58:18.986829: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 29, 40, 4)         104       
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 40, 4)         404       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 20, 4)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 20, 4)         0         
                                                                 
 flatten (Flatten)           (None, 1120)              0         
                                                                 
 dense (Dense)               (None, 512)               573952    
                                                        

In [3]:
with open(LABELS_PATH) as f:
    label_map = yaml.load(f, Loader=yaml.SafeLoader)["labels"]
reverse_label_map = {v:k for k,v in label_map.items()}
repr(reverse_label_map)

image_files = []
IMAGES_DIR="images/esp32/live/discard"
for dir,_,files in os.walk(IMAGES_DIR):
    for file in files:
        image_files.append(os.path.join(dir, file))
print(len(image_files))
print(image_files[0])

604
images/esp32/live/discard/d48939b4-7307-4d02-813c-e0f84b60dbb3.png


In [4]:
def predict(image):
    img_array = np.asarray(image, dtype=np.uint8).reshape(29,40, 1)
    formatted_img_array = np.expand_dims(img_array, axis=0)

    output_data = model.predict(formatted_img_array, verbose=0)
    raw_pred_prob = np.max(output_data)
    raw_pred = tf.argmax(output_data, 1).numpy()[0]

    return (raw_pred, raw_pred_prob)

In [5]:
random_path = random.sample(image_files, k=1)[0]
live_label = random_path.split(os.path.sep)[2]
print(random_path)
i = Image.open(random_path).point(lambda x: 255 if x>0 else 0)
pred = predict(i)
print(f"{reverse_label_map[pred[0]]} - {pred[1]}")
if pred[1] < .95:
    status = "DISCARD"
else:
    status = "KEEP"
print(f"live: {live_label} - local: {reverse_label_map[pred[0]]} ({status})")
display(i)

images/esp32/live/discard/c7dcd4ae-fb56-45c2-872f-b03864892983.png


2023-05-29 20:58:22.302254: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700


descendo - 0.15057262778282166
live: live - local: descendo (DISCARD)


In [6]:
THRESH = .95
images = []
image_arrays = []
for imgfile in image_files:
    i = Image.open(imgfile).point(lambda x: 255 if x>0 else 0)
    images.append(i)
    img_array = np.asarray(i, dtype=np.uint8).reshape(29,40, 1)
    image_arrays.append(img_array)

all_img_arrays = np.array(image_arrays)
preds = model.predict(all_img_arrays)
max_arg = np.argmax(preds, axis=1)
max_value = np.amax(preds, axis=1)

kept_idxs = np.nonzero(max_value > THRESH)[0]
print(f"Number of kept images: {len(kept_idxs)}")
for idx in kept_idxs:
    print(f"{reverse_label_map[max_arg[idx]]}: {max_value[idx]}")
    display(images[idx])


19/19 [==============================] - 0s 3ms/step
Number of kept images: 65
descendo: 0.9628841280937195


mimblewimble: 0.9902992248535156


mimblewimble: 0.9831879734992981


mimblewimble: 0.9908457398414612


mimblewimble: 0.9999803304672241


...

mimblewimble: 0.9999973773956299


mimblewimble: 0.9995317459106445


incendio: 0.9917580485343933


incendio: 0.9908185601234436


descendo: 0.9597105383872986
